- incorporate data augmentation for cases where output size is larger than input size and output size is fixed
- delete method2

In [1]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
from pathlib import Path
from collections import defaultdict
from itertools import product
from matplotlib import colors
import matplotlib.pyplot as plt
import numpy as np
from itertools import combinations,permutations
from sklearn.tree import *
from sklearn import tree
from sklearn.ensemble import BaggingClassifier
import random
from math import floor
import os

# method

In [2]:
data_path = Path("/kaggle/input/abstraction-and-reasoning-challenge")
train_path = data_path/'training'
eval_path = data_path/'evaluation'
test_path = data_path/'test'

In [3]:
def preprocess(task, index): # modify mistakes in data
    # 025d127b
    if index == "025d127b.json":
        for i in range(9, 12):
            for j in range(3, 8):
                task['train'][0]['output'][i][j] = 0
        for i in range(7, 10):
            for j in range(3, 6):
                task['train'][0]['output'][i][j] = 2
        task['train'][0]['output'][8][4] = 0
    # ef135b50
    elif index == "ef135b50.json":
        task['test'][0]['output'][6][4] = 9
    # bd14c3bf
    elif index == "bd14c3bf.json":
        for i in range(3):
            for j in range(5):
                if task['test'][0]['input'][i][j] == 1:
                    task['test'][0]['input'][i][j] = 2
    # a8610ef7
    elif index == "a8610ef7.json":
        for i in range(6):
            for j in range(6):
                if task['test'][0]['output'][i][j] == 8:
                    task['test'][0]['output'][i][j] = 5
        task['train'][3]['input'][0][1] = 2
        task['train'][3]['input'][5][1] = 2
    # 54db823b
    elif index == "54db823b.json":
        task['train'][0]['output'][2][3] = 3
        task['train'][0]['output'][2][4] = 9
    # e5062a87
    elif index == "e5062a87.json":
        for j in range(3, 7):
            task['train'][1]['output'][1][j] = 2
    # 1b60fb0c
    elif index == "1b60fb0c.json":
        task['train'][1]['output'][8][8] = 0
        task['train'][1]['output'][8][9] = 0
    # 82819916
    elif index == "82819916.json":
        task['train'][0]['output'][4][5] = 4
    # fea12743
    elif index == "fea12743.json":
        for i in range(11, 16):
            for j in range(6):
                if task['train'][0]['output'][i][j] == 2:
                    task['train'][0]['output'][i][j] = 8
    # 42a50994
    elif index == "42a50994.json":
        task['train'][0]['output'][1][0] = 8
        task['train'][0]['output'][0][1] = 8
    # f8be4b64
    elif index == "f8be4b64.json":
        for j in range(19):
            if task['test'][0]['output'][12][j] == 0:
                task['test'][0]['output'][12][j] = 1
        task['test'][0]['output'][12][8] = 0
    # d511f180
    elif index == "d511f180.json":
        task['train'][1]['output'][2][2] = 9
    # 10fcaaa3
    elif index == "10fcaaa3.json":
        task['train'][1]['output'][4][7] = 8
    # cbded52d
    elif index == "cbded52d.json":
        task['train'][0]['input'][4][6] = 1
    # 11852cab
    elif index == "11852cab.json":
        task['train'][0]['input'][1][2] = 3
    # 868de0fa
    elif index == "868de0fa.json":
        for j in range(2, 9):
            task['train'][2]['input'][9][j] = 0
            task['train'][2]['input'][10][j] = 1
            task['train'][2]['input'][15][j] = 0
            task['train'][2]['input'][16][j] = 1
        task['train'][2]['input'][15][2] = 1
        task['train'][2]['input'][15][8] = 1
    # 6d58a25d
    elif index == "6d58a25d.json":
        task['train'][0]['output'][10][0] = 0
        task['train'][2]['output'][6][13] = 4
    # a9f96cdd
    elif index == "a9f96cdd.json":
        task['train'][3]['output'][1][3] = 0
    # 48131b3c
    elif index == "48131b3c.json":
        task['train'][2]['output'][4][4] = 0
    # 150deff5
    elif index == "150deff5.json":
        aux = task['train'][2]['output'].copy()
        task['train'][2]['output'] = task['train'][2]['input'].copy()
        task['train'][2]['input'] = aux
    # 17cae0c1
    elif index == "17cae0c1.json":
        for i in range(3):
            for j in range(3, 6):
                task['test'][0]['output'][i][j] = 9
    # e48d4e1a
    elif index == "e48d4e1a.json":
        task['train'][3]['input'][0][9] = 5
        task['train'][3]['output'][0][9] = 0
    # 8fbca751
    elif index == "8fbca751.json":
        task['train'][1]['output'][1][3] = 2
        task['train'][1]['output'][2][3] = 8
    # 4938f0c2
    elif index == "4938f0c2.json":
        for i in range(12):
            for j in range(6,13):
                if task['train'][2]['input'][i][j]==2:
                    task['train'][2]['input'][i][j] = 0
        for i in range(5,11):
            for j in range(7):
                if task['train'][2]['input'][i][j]==2:
                    task['train'][2]['input'][i][j] = 0
    # 9aec4887
    elif index == "9aec4887.json":
        task['train'][0]['output'][1][4] = 8
    # b0f4d537
    elif index == "b0f4d537.json":
        for i in range(9):
            task['train'][0]['output'][i][3] = 0
            task['train'][0]['output'][i][4] = 1
        task['train'][0]['output'][2][3] = 3
        task['train'][0]['output'][2][4] = 3
        task['train'][0]['output'][5][3] = 2
    # aa300dc3
    elif index == "aa300dc3.json":
        task['train'][1]['input'][1][7] = 5
        task['train'][1]['output'][1][7] = 5
        task['train'][1]['input'][8][2] = 5
        task['train'][1]['output'][8][2] = 5
    # ad7e01d0
    elif index == "ad7e01d0.json":
        task['train'][0]['output'][6][7] = 0
    # a8610ef7
    elif index == "a8610ef7.json":
        task['train'][3]['input'][0][1] = 0
        task['train'][3]['input'][5][1] = 0
        task['train'][3]['output'][0][1] = 0
        task['train'][3]['output'][5][1] = 0
    # 97239e3d
    elif index == "97239e3d.json":
        task['test'][0]['input'][14][6] = 0
        task['test'][0]['input'][14][10] = 0
    # d687bc17
    elif index == "d687bc17.json":
        task['train'][2]['output'][7][1] = 4
    return task

In [4]:
def color_check(color_list):
    tmp = color_list[0]
    for i in range(1,len(color_list)):
        if set(tmp) != set(color_list[i]):
            return False
    return True

def create_df(folder_path):
    task_names_list = sorted(os.listdir(folder_path))
    task_list = []
    for task_name in task_names_list: 
        task_file = str(folder_path / task_name)
        with open(task_file, 'r') as f:
            task = json.load(f)
            if "test" not in str(folder_path):
                task = preprocess(task, task_name)
            task_list.append(task)
    
    df = pd.DataFrame()
    df['task_name'] = task_names_list
    df['task'] = task_list
    df['number_of_train_pairs'] = df['task'].apply(lambda x: len(x['train']))
    df['number_of_test_pairs'] = df['task'].apply(lambda x: len(x['test']))
    
    # Compare image sizes
    df['inputs_all_have_same_height'] = df['task'].apply(
        lambda task: int(len(set([len(example['input']) for example in task['train']])) == 1)
    )
    df['inputs_all_have_same_width'] = df['task'].apply(
        lambda task: int(len(set([len(example['input'][0]) for example in task['train']])) == 1)
    )
    df['inputs_all_have_same_shape'] = df['inputs_all_have_same_height'] * df['inputs_all_have_same_width']
    df['input_height_if_constant'] = df['task'].apply(
        lambda task: len(task['train'][0]['input'])
                     if (len(set([len(example['input']) for example in task['train']])) == 1)
                     else np.nan
    )
    df['input_width_if_constant'] = df['task'].apply(
        lambda task: len(task['train'][0]['input'][0])
                     if (len(set([len(example['input'][0]) for example in task['train']])) == 1)
                     else np.nan
    )
    df['outputs_all_have_same_height'] = df['task'].apply(
        lambda task: int(len(set([len(example['output']) for example in task['train']])) == 1)
    )
    df['outputs_all_have_same_width'] = df['task'].apply(
        lambda task: int(len(set([len(example['output'][0]) for example in task['train']])) == 1)
    )
    df['outputs_all_have_same_shape'] = df['outputs_all_have_same_height'] * df['outputs_all_have_same_width']
    df['output_height_if_constant'] = df['task'].apply(
        lambda task: len(task['train'][0]['output'])
                     if (len(set([len(example['output']) for example in task['train']])) == 1)
                     else np.nan
    )
    df['output_width_if_constant'] = df['task'].apply(
        lambda task: len(task['train'][0]['output'][0])
                     if (len(set([len(example['output'][0]) for example in task['train']])) == 1)
                     else np.nan
    )  
    df['in_each_pair_shape_doesnt_change'] = df['task'].apply(
        lambda task: np.prod([int(len(example['input'][0])==len(example['output'][0])
                                  and len(example['input'])==len(example['output'])
                                 ) for example in task['train']
                            ])
    )
    df['in_each_pair_shape_ratio_is_the_same'] = df['task'].apply(
        lambda task: (len(set([len(example['input'][0]) / len(example['output'][0])
                                 for example in task['train']]))==1) * (
                      len(set([len(example['input']) / len(example['output'])
                                 for example in task['train']]))==1)
    )
    df['o/i_height_ratio_if_constant'] = df['task'].apply(
        lambda task: len(task['train'][0]['output']) / len(task['train'][0]['input'])
                     if (len(set([len(example['input']) / len(example['output'])
                                 for example in task['train']]))==1)
                     else np.nan
    )
    df['o/i_width_ratio_if_constant'] = df['task'].apply(
        lambda task: len(task['train'][0]['output'][0]) / len(task['train'][0]['input'][0])
                     if (len(set([len(example['input'][0]) / len(example['output'][0])
                                 for example in task['train']]))==1)
                     else np.nan
    )
    
    # my idea ---------
    df["same_color_sum"] = df['task'].apply(lambda task: 
                        np.all([int(sum(sum(np.array(example['input'])))== sum(sum(np.array(example['output'])))) for example in task['train']]))
    
    df["same_color_sum_in_edge"] = df['task'].apply(lambda task: 
                        np.all([int(sum(np.array(example['input'])[0,:]) +sum(np.array(example['input'])[:,0]) + 
                                    sum(np.array(example['input'])[-1,:]) +sum(np.array(example['input'])[:,-1])
                                    == 
                                    sum(np.array(example['output'])[0,:]) +sum(np.array(example['output'])[:,0]) + 
                                    sum(np.array(example['output'])[-1,:]) +sum(np.array(example['output'])[:,-1])) for example in task['train']]))
    
    df["io_color_kind_diff"] = df['task'].apply(lambda task: [len(np.unique(np.array(example['input']))) - len(np.unique(np.array(example['output']))) for example in task['train']])
    df["io_color_kind_diff_constant"] = df['io_color_kind_diff'].apply(lambda task: np.unique(np.array(task))[0] if len(np.unique(np.array(task)))==1 else -1)
    df["output_not_include_0"] = df['task'].apply(lambda task: np.all([np.all(np.array(example['output']) > 0) for example in task['train']]))
    df["increase_color_sum"] = df['task'].apply(lambda task: 
                        np.all([int(sum(sum(np.array(example['input']))) < sum(sum(np.array(example['output'])))) for example in task['train']]))
    df["decrease_color_sum"] = df['task'].apply(lambda task: 
                        np.all([int(sum(sum(np.array(example['input']))) > sum(sum(np.array(example['output'])))) for example in task['train']]))
    # **
    df["input_color_change_or_not"] = df['task'].apply(lambda task: color_check([list(np.unique(np.array(example['input']))) for example in task["train"]] ))
    # **
    df['color_kind_increase'] = df['task'].apply(
        lambda task: np.all([len(np.unique(np.array(example['input']))) < len(np.unique(np.array(example['output']))) for example in task['train']]))
    df['color_kind_decrease'] = df['task'].apply(
        lambda task: np.all([len(np.unique(np.array(example['input']))) > len(np.unique(np.array(example['output']))) for example in task['train']]))
    df['smaller_output'] = df['task'].apply(
        lambda task: np.all([(np.array(example['input']).shape[0] >= np.array(example['output']).shape[0] and 
                             np.array(example['input']).shape[1] >= np.array(example['output']).shape[1] and 
                             (np.array(example['input']).shape[0] > np.array(example['output']).shape[0] or 
                             np.array(example['input']).shape[1] > np.array(example['output']).shape[1])) for example in task['train']]))
    df['larger_output'] = df['task'].apply(
        lambda task: np.all([(np.array(example['input']).shape[0] <= np.array(example['output']).shape[0] and 
                             np.array(example['input']).shape[1] <= np.array(example['output']).shape[1] and 
                             (np.array(example['input']).shape[0] < np.array(example['output']).shape[0] or 
                             np.array(example['input']).shape[1] < np.array(example['output']).shape[1])) for example in task['train']]))
    return df

training_descriptive_df = create_df(train_path)
evaluation_descriptive_df = create_df(eval_path)
test_descriptive_df = create_df(test_path)

In [5]:
def classification(row):
    # same shape and color doesn't change in input and color kind decrease
    if row["in_each_pair_shape_doesnt_change"] == 1 and row["o/i_height_ratio_if_constant"] ==1 and row["o/i_width_ratio_if_constant"]==1 and row.input_color_change_or_not==1 and row.color_kind_decrease==1:
        return 1
    # same shape and color doesn't change in input and color kind increase
    elif row["in_each_pair_shape_doesnt_change"] == 1 and row["o/i_height_ratio_if_constant"] ==1 and row["o/i_width_ratio_if_constant"]==1 and row.input_color_change_or_not==1 and row.color_kind_increase==1:
        return 2
    # same shape and color doesn't change in input and color kind same
    elif row["in_each_pair_shape_doesnt_change"] == 1 and row["o/i_height_ratio_if_constant"] ==1 and row["o/i_width_ratio_if_constant"]==1 and row.input_color_change_or_not==1 and row.color_kind_increase==0  and row.color_kind_decrease==0:
        return 3
    # same shape and decrease color sum → xgboost
    if row["in_each_pair_shape_doesnt_change"] == 1 and row["o/i_height_ratio_if_constant"] ==1 and row["o/i_width_ratio_if_constant"]==1 and row.input_color_change_or_not==0 and row.color_kind_decrease==1:
        return 4
    # different shape and decrease color sum
    if row["in_each_pair_shape_doesnt_change"] == 1 and row["o/i_height_ratio_if_constant"] ==1 and row["o/i_width_ratio_if_constant"]==1 and row.input_color_change_or_not==0 and row.color_kind_increase==1:
        return 5
    # different shape and increase color sum
    elif row["in_each_pair_shape_doesnt_change"] == 1 and row["o/i_height_ratio_if_constant"] ==1 and row["o/i_width_ratio_if_constant"]==1 and row.input_color_change_or_not==0 and row.color_kind_increase==0  and row.color_kind_decrease==0:
        return 6
    # otherwise
    elif row["smaller_output"] == 1 and row["outputs_all_have_same_shape"] == 1:
        return 7
    elif row["larger_output"] == 1 and row["outputs_all_have_same_shape"] == 1:
        return 8
    else:
        return 9

training_descriptive_df["class"] = training_descriptive_df.apply(lambda x: classification(x), axis=1)
evaluation_descriptive_df["class"] = evaluation_descriptive_df.apply(lambda x: classification(x), axis=1)
test_descriptive_df["class"] = test_descriptive_df.apply(lambda x: classification(x), axis=1)

In [6]:
def plot_result(inp,eoup,oup):
    """
    Plots the first train and test pairs of a specified task,
    using same color scheme as the ARC app
    """
    cmap = colors.ListedColormap(
        ['#000000', '#0074D9','#FF4136','#2ECC40','#FFDC00',
         '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25'])
    norm = colors.Normalize(vmin=0, vmax=9)
    fig, axs = plt.subplots(1, 3, figsize=(15,15))
    
    axs[0].imshow(inp, cmap=cmap, norm=norm)
    axs[0].axis('off')
    axs[0].set_title('Input')

    axs[1].imshow(eoup, cmap=cmap, norm=norm)
    axs[1].axis('off')
    axs[1].set_title('Output')
    
    axs[2].imshow(oup, cmap=cmap, norm=norm)
    axs[2].axis('off')
    axs[2].set_title('Model prediction')
    
    plt.grid()
    plt.tight_layout()
    plt.show()

def plot_mats(mats):
    cmap = colors.ListedColormap(
        ['#000000', '#0074D9','#FF4136','#2ECC40','#FFDC00',
         '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25'])
    norm = colors.Normalize(vmin=0, vmax=9)
    fig, axs = plt.subplots(1, len(mats), figsize=(15,15))
    
    for i in range(len(mats)):
        axs[i].imshow(mats[i], cmap=cmap, norm=norm)
        axs[i].axis('off')
        axs[i].set_title('Fig: '+str(i))
    
    plt.rc('grid', linestyle="-", color='white')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [7]:
def getiorc(pair):
    inp = pair["input"]
    return pair["input"],pair["output"],len(inp),len(inp[0])
    
def getAround(i,j,inp,size=1):
    #v = [-1,-1,-1,-1,-1,-1,-1,-1,-1]
    r,c = len(inp),len(inp[0])
    v = []
    sc = [0]
    for q in range(size):
        sc.append(q+1)
        sc.append(-(q+1))
    for idx,(x,y) in enumerate(product(sc,sc)):
        ii = (i+x)
        jj = (j+y)
        v.append(-1)
        if((0<= ii < r) and (0<= jj < c)):
            v[idx] = (inp[ii][jj])
    return v

def getDiagonal(i,j,r,c):
    return
        
    
def getX(inp,i,j,size):
    z = []
    n_inp = np.array(inp)
    z.append(i)
    z.append(j)
    r,c = len(inp),len(inp[0])
    for m in range(5):
        z.append(i%(m+1))
        z.append(j%(m+1))
    z.append(i+j)
    z.append(i*j)
#     z.append(i%j)
#     z.append(j%i)
    z.append((i+1)/(j+1))
    z.append((j+1)/(i+1))
    z.append(r)
    z.append(c)
    z.append(len(np.unique(n_inp[i,:])))
    z.append(len(np.unique(n_inp[:,j])))
    arnd = getAround(i,j,inp,size)
    z.append(len(np.unique(arnd)))
    z.extend(arnd)
    return z

def getXy(inp,oup,size):
    x = []
    y = []
    r,c = len(inp),len(inp[0])
    for i in range(r):
        for j in range(c):
            x.append(getX(inp,i,j,size))
            y.append(oup[i][j])
    return x,y
    
def getBkgColor(task_json):
    color_dict = defaultdict(int)
    
    for pair in task_json['train']:
        inp,oup,r,c = getiorc(pair)
        for i in range(r):
            for j in range(c):
                color_dict[inp[i][j]]+=1
    color = -1
    max_count = 0
    for col,cnt in color_dict.items():
        if(cnt > max_count):
            color = col
            max_count = cnt
    return color

In [8]:
def padding(tasks):
    ans = []
    for task in tasks:
        tmp = {"input": 0, "output": 0}
        tmp["input"] = task["input"]
        saigen = np.zeros(np.array(task["input"]).shape)
        cur_h, cur_w = np.array(task["output"]).shape
        saigen[:cur_h, :cur_w] = np.array(task["output"])
        tmp["output"] = saigen.astype(int).tolist()
        ans.append(tmp)
    return ans, cur_h, cur_w

def input_padding(tasks):
    ans = []
    for task in tasks:
        tmp = {"input": 0, "output": 0}
        tmp["output"] = task["output"]
        saigen = np.zeros(np.array(task["output"]).shape)
        cur_h, cur_w = np.array(task["input"]).shape
        saigen[:cur_h, :cur_w] = np.array(task["input"])
        tmp["input"] = saigen.astype(int).tolist()
        ans.append(tmp)
    return ans

In [9]:
def get_num_colors(inp,oup,bl_cols):
    r,c = len(inp),len(inp[0])
    return 

def replace(inp,uni,perm):
    # uni = '234' perm = ['5','7','9']
    #print(uni,perm)
    r_map = { int(c):int(s) for c,s in zip(uni,perm)}
    r,c = len(inp),len(inp[0])
    rp = np.array(inp).tolist()
    #print(rp)
    for i in range(r):
        for j in range(c):
            if(rp[i][j] in r_map):
                rp[i][j] = r_map[rp[i][j]]
    return rp
            
    
def augment(inp,oup,bl_cols):
    cols = "0123456789"
    npr_map = [1,9,72,3024,15120,60480,181440,362880,362880]
    uni = "".join([str(x) for x in np.unique(inp).tolist()])
    for c in bl_cols:
        cols=cols.replace(str(c),"")
        uni=uni.replace(str(c),"")

    exp_size = len(inp)*len(inp[0])*npr_map[len(uni)]
    
    mod = floor(exp_size/120000)
    mod = 1 if mod==0 else mod
    
    #print(exp_size,mod,len(uni))
    result = []
    count = 0
    for comb in combinations(cols,len(uni)):
        for perm in permutations(comb):
            count+=1
            if(count % mod == 0):
                result.append((replace(inp,uni,perm),replace(oup,uni,perm)))
    return result
            
def get_flips(inp,oup):
    result = []
    n_inp = np.array(inp)
    n_oup = np.array(oup)
    result.append((np.fliplr(inp).tolist(),np.fliplr(oup).tolist()))
    result.append((np.rot90(np.fliplr(inp),1).tolist(),np.rot90(np.fliplr(oup),1).tolist()))
    result.append((np.rot90(np.fliplr(inp),2).tolist(),np.rot90(np.fliplr(oup),2).tolist()))
    result.append((np.rot90(np.fliplr(inp),3).tolist(),np.rot90(np.fliplr(oup),3).tolist()))
    result.append((np.flipud(inp).tolist(),np.flipud(oup).tolist()))
    result.append((np.rot90(np.flipud(inp),1).tolist(),np.rot90(np.flipud(oup),1).tolist()))
    result.append((np.rot90(np.flipud(inp),2).tolist(),np.rot90(np.flipud(oup),2).tolist()))
    result.append((np.rot90(np.flipud(inp),3).tolist(),np.rot90(np.flipud(oup),3).tolist()))
    result.append((np.fliplr(np.flipud(inp)).tolist(),np.fliplr(np.flipud(oup)).tolist()))
    result.append((np.flipud(np.fliplr(inp)).tolist(),np.flipud(np.fliplr(oup)).tolist()))
    return result
    
def gettaskxy(task_json,aug,around_size,bl_cols,flip=True):    
    X = []
    Y = []
    for pair in task_json['train']:
        inp,oup=pair["input"],pair["output"]
        tx,ty = getXy(inp,oup,around_size)
        X.extend(tx)
        Y.extend(ty)
        if(flip):
            for ainp,aoup in get_flips(inp,oup):
                tx,ty = getXy(ainp,aoup,around_size)
                X.extend(tx)
                Y.extend(ty)
                if(aug):
                    augs = augment(ainp,aoup,bl_cols)
                    for ainp,aoup in augs:
                        tx,ty = getXy(ainp,aoup,around_size)
                        X.extend(tx)
                        Y.extend(ty)
        if(aug):
            augs = augment(inp,oup,bl_cols)
            for ainp,aoup in augs:
                tx,ty = getXy(ainp,aoup,around_size)
                X.extend(tx)
                Y.extend(ty)
    return X,Y

def test_predict(task_json,model,size):
    inp = task_json['test'][0]['input']
    eoup = task_json['test'][0]['output']
    r,c = len(inp),len(inp[0])
    oup = predict(inp,model,size)
    return inp,eoup,oup

def predict(inp,model,size):
    r,c = len(inp),len(inp[0])
    oup = np.zeros([r,c],dtype=int)
    for i in range(r):
        for j in range(c):
            x = getX(inp,i,j,size)
            o = int(model.predict([x]))
            o = 0 if o<0 else o
            oup[i][j]=o
    return oup

def submit_predict(task_json,model,size):
    pred_map = {}
    idx=0
    for pair in task_json['test']:
        inp = pair["input"]
        oup = predict(inp,model,size)
        pred_map[idx] = oup.tolist()
        idx+=1
        #plot_result(inp,oup,oup)
    return pred_map

def dumb_predict(task_json):
    pred_map = {}
    idx=0
    for pair in task_json['test']:
        inp = pair["input"]
        pred_map[idx] = [[0,0],[0,0]]
        idx+=1
    return pred_map
    

In [10]:
def get_loss(model,task_json,size):
    total = 0
    for pair in task_json['train']:
        inp,oup=pair["input"],pair["output"]
        eoup = predict(inp,model,size)
        total+= np.sum((np.array(oup) != np.array(eoup)))
    return total

def get_test_loss(model,task_json,size):
    total = 0
    for pair in task_json['test']:
        inp,oup=pair["input"],pair["output"]
        eoup = predict(inp,model,size)
        total+= np.sum((np.array(oup) != np.array(eoup)))
    return total

def get_a_size(task_json):
    return 4;

def get_bl_cols(task_json):
    result = []
    bkg_col = getBkgColor(task_json);
    result.append(bkg_col)
    # num_input,input_cnt,num_output,output_cnt
    met_map = {}
    for i in range(10):
        met_map[i] = [0,0,0,0]
        
    total_ex = 0
    for pair in task_json['train']:
        inp,oup=pair["input"],pair["output"]
        u,uc = np.unique(inp, return_counts=True)
        inp_cnt_map = dict(zip(u,uc))
        u,uc = np.unique(oup, return_counts=True)
        oup_cnt_map = dict(zip(u,uc))
        
        for col,cnt in inp_cnt_map.items():
            met_map[col][0] = met_map[col][0] + 1
            met_map[col][1] = met_map[col][1] + cnt
        for col,cnt in oup_cnt_map.items():
            met_map[col][2] = met_map[col][2] + 1
            met_map[col][3] = met_map[col][3] + cnt
        total_ex+=1
    
    for col,met in met_map.items():
        num_input,input_cnt,num_output,output_cnt = met
        if(num_input == total_ex or num_output == total_ex):
            result.append(col)
        elif(num_input == 0 and num_output > 0):
            result.append(col)
    
    result = np.unique(result).tolist()
    if(len(result) == 10):
        result.append(bkg_col)
    return np.unique(result).tolist()


In [11]:
def flattener(pred):
    str_pred = str([row for row in pred])
    str_pred = str_pred.replace(', ', '')
    str_pred = str_pred.replace('[[', '|')
    str_pred = str_pred.replace('][', '|')
    str_pred = str_pred.replace(']]', '|')
    return str_pred

def combine_preds(tid,pm1,pm2):
    result = []
    for i in range(len(pm1)):
        tk_s = tid+"_"+str(i)
        str_pred = flattener(pm1[i])+" "+flattener(pm2[i])
        #print(tk_s,str_pred)
        result.append([tk_s,str_pred])
    return result
        

In [12]:
def inp_oup_dim_same(task_json):
    return all([ len(pair["input"]) == len(pair["output"]) and len(pair["input"][0]) == len(pair["output"][0])
                for pair in task_json['train']])
    
solved_task = 0
total_task = 0
task_ids = []
task_preds = []
for task_path in test_path.glob("*.json"):
    task_json = json.load(open(task_path))
    tk_id = str(task_path).split("/")[-1].split(".")[0]
    print(tk_id+".json")
    class_num = test_descriptive_df[test_descriptive_df.task_name == tk_id+".json"]["class"].values[0]
    #if(inp_oup_dim_same(task_json)):
    if class_num <= 6:
        a_size = get_a_size(task_json)
        bl_cols = get_bl_cols(task_json)
        
        isflip = False
        X1,Y1 = gettaskxy(task_json,True,1,bl_cols,isflip)
        X3,Y3 = gettaskxy(task_json,True,3,bl_cols,isflip)
        X5,Y5 = gettaskxy(task_json,True,5,bl_cols,isflip)
        
        model_1 = BaggingClassifier(base_estimator=DecisionTreeClassifier(),n_estimators=100).fit(X1, Y1)
        #model_3 = BaggingClassifier(base_estimator=DecisionTreeClassifier(),n_estimators=100).fit(X3, Y3)
        model_5 = BaggingClassifier(base_estimator=DecisionTreeClassifier(),n_estimators=100).fit(X5, Y5)
        
        pred_map_1 = submit_predict(task_json,model_1,1)
        #pred_map_3 = submit_predict(task_json,model_3,3)
        pred_map_5 = submit_predict(task_json,model_5,5)
        
        for tks,str_pred in combine_preds(tk_id,pred_map_1,pred_map_5):
            task_ids.append(tks)
            task_preds.append(str_pred)
        solved_task+=1
        
    #elif class_num == 7:
    #    task_json["train"] = padding(task_json["train"])
        
    #    a_size = get_a_size(task_json)
    #    bl_cols = get_bl_cols(task_json)
        
    #    isflip = False
    #    X1,Y1 = gettaskxy(task_json,True,1,bl_cols,isflip)
    #    X3,Y3 = gettaskxy(task_json,True,3,bl_cols,isflip)
    #    X5,Y5 = gettaskxy(task_json,True,5,bl_cols,isflip)
        
    #    model_1 = BaggingClassifier(base_estimator=DecisionTreeClassifier(),n_estimators=100).fit(X1, Y1)
        #model_3 = BaggingClassifier(base_estimator=DecisionTreeClassifier(),n_estimators=100).fit(X3, Y3)
    #    model_5 = BaggingClassifier(base_estimator=DecisionTreeClassifier(),n_estimators=100).fit(X5, Y5)
        
    #    pred_map_1 = submit_predict(task_json,model_1,1)
        #pred_map_3 = submit_predict(task_json,model_3,3)
    #    pred_map_5 = submit_predict(task_json,model_5,5)
        
    #    for tks,str_pred in combine_preds(tk_id,pred_map_1,pred_map_5):
    #        task_ids.append(tks)
    #        task_preds.append(str_pred)
    #    solved_task+=1

    elif class_num == 8:
        task_json["train"] = input_padding(task_json["train"])
        task_json["test"] = input_padding(task_json["test"])
        
        a_size = get_a_size(task_json)
        bl_cols = get_bl_cols(task_json)
        
        isflip = False
        X1,Y1 = gettaskxy(task_json,True,1,bl_cols,isflip)
        X3,Y3 = gettaskxy(task_json,True,3,bl_cols,isflip)
        X5,Y5 = gettaskxy(task_json,True,5,bl_cols,isflip)
        
        model_1 = BaggingClassifier(base_estimator=DecisionTreeClassifier(),n_estimators=100).fit(X1, Y1)
        #model_3 = BaggingClassifier(base_estimator=DecisionTreeClassifier(),n_estimators=100).fit(X3, Y3)
        model_5 = BaggingClassifier(base_estimator=DecisionTreeClassifier(),n_estimators=100).fit(X5, Y5)
        
        pred_map_1 = submit_predict(task_json,model_1,1)
        #pred_map_3 = submit_predict(task_json,model_3,3)
        pred_map_5 = submit_predict(task_json,model_5,5)
        
        for tks,str_pred in combine_preds(tk_id,pred_map_1,pred_map_5):
            task_ids.append(tks)
            task_preds.append(str_pred)
        solved_task+=1
        
    else:
        pred_map_1 = dumb_predict(task_json)
        #pred_map_3 = dumb_predict(task_json)
        pred_map_5 = dumb_predict(task_json)
        
        for tks,str_pred in combine_preds(tk_id,pred_map_1,pred_map_5):
            task_ids.append(tks)
            task_preds.append(str_pred)
            #print(tks,str_pred)
        
    total_task+=1

19bb5feb.json
2037f2c7.json
11e1fe23.json
332efdb3.json
136b0064.json
1c56ad9f.json
40f6cd08.json
292dd178.json
12997ef3.json
25094a63.json
0bb8deee.json
2072aba6.json


KeyError: 'output'

In [13]:
sample_sub1 = pd.DataFrame({"output_id":task_ids,'output':task_preds})
sample_sub1.to_csv("submission.csv", index=None)

In [14]:
#sample_sub2 = pd.DataFrame({"output_id":task_ids2,'output':task_preds2})

#final_sub = sample_sub1
#final_sub = final_sub.sort_values(by="output_id").reset_index(drop=True)

#sample_sub2 = sample_sub2.sort_values(by="output_id")
#out1 = final_sub["output"].astype(str).values
#out2 = sample_sub2["output"].astype(str).values

#merge_output = []
#for o1, o2 in zip(out1, out2):
#    o = o1.strip().split(" ")[:2] + o2.strip().split(" ")[:1]
#    o = " ".join(o[:3])
#    merge_output.append(o)
    
#final_sub["output"] = merge_output
#final_sub["output"] = final_sub["output"].astype(str)
#final_sub.to_csv("submission.csv", index=False)